In [1]:
import pandas as pd
import numpy as np
import io
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [2]:
filename_anym = '../antonym-synonym set/Antonym_vietnamese.txt'

In [3]:
with open(filename_anym,'r', encoding='utf-8') as f:
    lines = [line.rstrip('\n') for line in f]

In [4]:
df = pd.DataFrame(columns= ['a','b','label'])
df

,a,b,label


In [5]:
i = 0
for line in lines:
    s = line.split()
    u1 = s[0].strip() # word 1
    u2 = s[1].strip() # word 2
    df.loc[i] = (u1, u2, 0)
    i += 1
    #df.append(df2, ignore_index=True)

In [6]:
df

,a,b,label
0,ác,hiền,0
1,ác,thiện,0
2,ác_cảm,thiện_cảm,0
3,ác_độc,hiền_lành,0
4,ác_độc,hiền_từ,0
...,...,...,...
1995,bẩn_thỉu,sạch_sẽ,0
1996,bận,rảnh,0
1997,bận,rảnh_rang,0
1998,bận,rảnh_rỗi,0


In [7]:
filename_sym = '../antonym-synonym set/Synonym_vietnamese.txt'

In [8]:
df_sym = pd.DataFrame(columns= ['a','b','label'])
with open(filename_sym,'r', encoding='utf-8') as f:
    lines1 = [line.rstrip('\n') for line in f]

In [9]:
for line in lines1:
    s = line.split()
    if len(s) >= 2:
        u1 = s[0].strip() # word 1
        u2 = s[1].strip() # word 2
        df_sym.loc[i] = (u1, u2, 1)
        i += 1
    #df.append(df2, ignore_index=True)
df_sample = df_sym.sample(2000, random_state=10)
df = df.append(df_sample)

C:\Users\Viet\AppData\Local\Temp\ipykernel_3560\2589219976.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_sample)


In [10]:
df

,a,b,label
0,ác,hiền,0
1,ác,thiện,0
2,ác_cảm,thiện_cảm,0
3,ác_độc,hiền_lành,0
4,ác_độc,hiền_từ,0
...,...,...,...
13491,xôn_xao,xôn_xang,1
5694,đằng_hắng,tằng_hắng,1
4183,chọc_giận,trêu_ngươi,1
2368,bảnh,điển,1


In [12]:
import hashlib
from typing import Tuple

from sklearn.base import BaseEstimator, TransformerMixin


def hash_modulo(val, mod):
    md5 = hashlib.md5()  # can be other deterministic hash functions
    md5.update(str(val).encode())
    return int(md5.hexdigest(), 16) % mod


class FeatureHasher(BaseEstimator, TransformerMixin):
    def __init__(self, num_buckets: int):
        self.num_buckets = num_buckets

    def fit(self, X: pd.Series):
        return self

    def transform(self, X: pd.Series):
        return X.apply(lambda x: hash_modulo(x, self.num_buckets))


In [13]:
fh = FeatureHasher(num_buckets=4000)
df['a'] = fh.fit_transform(df['a'])
df['b'] = fh.transform(df['b'])

In [15]:
X = df.drop(columns=['label'])
y = df['label']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)

In [16]:
clf = RandomForestClassifier()
clf.fit(X, y)

RandomForestClassifier()

In [17]:
filename_test1 = '../datasets/ViCon-400/400_noun_pairs.txt'
filename_test2 = '../datasets/ViCon-400/400_verb_pairs.txt'
filename_test3 = '../datasets/ViCon-400/600_adj_pairs.txt'
df_test = pd.DataFrame(columns= ['a','b','label'])
i = 0
def get_Test(filename):
    global i
    with open(filename,'r', encoding='utf-8') as f:
        lines = [line.rstrip('\n') for line in f]
    for line in lines:
        s = line.split()
        if s[0] != 'Word1':
            u1 = s[0].strip() # word 1
            u2 = s[1].strip() # word 2
            u3 = s[2].strip()
            if u3 == 'ANT':
                label = 0
            else:
                label = 1
            df_test.loc[i] = (u1, u2, label)
            i += 1
get_Test(filename_test1)
get_Test(filename_test2)
get_Test(filename_test3)

In [19]:
df_test['a'] = fh.transform(df_test['a'])
df_test['b'] = fh.transform(df_test['b'])

In [20]:
X_test = df_test.drop(columns=['label'])
y_test = df_test['label']

In [21]:
pred = clf.predict(X_test)
accuracy_score(pred, y_test)

0.8614285714285714

In [22]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.78      1.00      0.88       700
           1       1.00      0.72      0.84       700

    accuracy                           0.86      1400
   macro avg       0.89      0.86      0.86      1400
weighted avg       0.89      0.86      0.86      1400

